# Cold Users

In [28]:
import os 
import pandas as pd  
import numpy as np
import math
import random
from IPython.display import clear_output
import tensorflow as tf
import time

In [29]:
user_following = np.load('user_following_127_57.npy')
all_3374 = np.load('all_2939D_y57.npy')
user_category = np.load('user_category_127.npy')
YouTuber_category = np.load('YouTuber_category_0.7_57.npy')
active_users = np.load('active_userID_127.npy')

In [30]:
user_test_amount = int(user_following.shape[0]*0.2)
yt_test_amount = 10
yt_num = user_following.shape[1]

In [31]:
def generate_train_test(user_following,feature,user_category,item_category,active_users,user_test_amount,yt_test_amount):
    print('Generate_train_test----')
    #The shape of orignal data
    print('user_following shape ',user_following.shape)
    print('feature shape ',feature.shape)
    print('user_category shape ',user_category.shape)
    print('YouTuber_category shape ',item_category.shape)
    user_category_norm = np.zeros(user_category.shape)
    for i in range(len(user_category)):
        user_category_norm[i] = user_category[i]/np.max(user_category[i])
    print('user_category after normalized by max...')
    print('user_category_norm shape ',user_category_norm.shape)
    
    #following youtuber for each user
    following_true = []
    for i in range(len(user_following)):
        each_user = []
        for j in range(len(user_following[i])):
            if user_following[i][j] == 1:
                each_user.append(j)
        following_true.append(each_user)
    #print(following_true)
    #number of followings for each user
    minlen = 10000
    maxlen = 0
    num_of_follower = []
    for i in range(len(following_true)):
        if len(following_true[i]) < minlen:
            minlen = len(following_true[i])
        if len(following_true[i]) > maxlen:
            maxlen = len(following_true[i])
        num_of_follower.append(len(following_true[i]))
    print('Min number of followings ',minlen)
    print('Max number of followings ',maxlen)
    
    num_of_users = user_following.shape[0]
    num_of_youtubers = item_category[0]
    
    """
    Spliting training and testing data
    """
    #testing user id random choice
    user_idx = [i for i in range(len(user_following))]
    random.seed(5)
    #choose test_id and sorted from small to large
    test_idx = sorted(random.sample(user_idx,user_test_amount))
    
    train_t = []
    train_f = []
    test_t = []
    test_f = []
    
    print('test_idx',test_idx)
    test_pos = -1
    for i in range(num_of_users):
        t_for_train = []
        f_for_train = []
        if i not in test_idx: #if not in test id, just append it to true or false list
            for j in range(yt_num):
                if user_following[i][j] == 1:
                    t_for_train.append(j)
                else:
                    f_for_train.append(j)
            train_t.append(t_for_train)
            train_f.append(f_for_train)

        else: #if in test id, choose 2 true and other 
            test_pos += 1
            temp_t = []
            temp_f = []
            for j in range(yt_num):
                if user_following[i][j] == 1:
                    temp_t.append(j)
                else:
                    temp_f.append(j)
            t_for_test = random.sample(temp_t,math.ceil(0.5*len(temp_t)))
            f_for_test  = random.sample(temp_f,yt_test_amount-len(t_for_test))

            test_t.append(t_for_test)
            test_f.append(f_for_test)

            #other for training
            t_for_train = [item for item in temp_t if not item in t_for_test]
            f_for_train = [item for item in temp_f if not item in f_for_test]
            train_t.append(t_for_train)
            train_f.append(f_for_train)
            
    # train_t[i] 代表的是user i positive feedback
    print('The length of train_t:',len(train_t))
    print('The length of train_f:',len(train_f))
    print('The length of test_t:',len(test_t))
    print('The length of test_f:',len(test_f))
    return train_t,train_f,test_t,test_f,user_category_norm

In [32]:
train_t,train_f,test_t,test_f,user_category_norm = generate_train_test(user_following,all_3374,user_category,YouTuber_category,active_users,user_test_amount,yt_test_amount)

Generate_train_test----
user_following shape  (127, 57)
feature shape  (57, 2939)
user_category shape  (127, 17)
YouTuber_category shape  (57, 17)
user_category after normalized by max...
user_category_norm shape  (127, 17)
Min number of followings  5
Max number of followings  5
test_idx [3, 6, 14, 20, 31, 32, 45, 47, 48, 59, 60, 67, 69, 79, 83, 88, 94, 99, 101, 107, 111, 117, 119, 120, 124]
The length of train_t: 127
The length of train_f: 127
The length of test_t: 25
The length of test_f: 25


In [33]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg)
print('比例:',avg/yt_test_amount)

training 4.409448818897638
testing 3.0
比例: 0.3


In [34]:
all_auxilary = [i for i in range(yt_num)]
n = len(user_following)
m = yt_num  
k = 64
l = all_3374.shape[1]
#embedding_dims = 150

In [35]:
def training(save_name): 
    
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    loss_acc_list = []
    t0=time.time()
    
    train_yes_id=[] 
    for q in range(7):
        print('Iteraction:',q)
        train_auc=0
        total_loss=0
        xuij_auc=0
        length = 0
        for z in range(n):
            """
            yes 用來存放選擇到的YouTuber feature (for auxilary)
            yesr 用來存放user對該YouTuber的喜好程度(user_category 跟 YouTuber_category的相似性)
            r_3 用來存放user 對該YouTuber種類的偏好(取max)
            """
            yes=[]
            yesr=[]
        
            """
            only choose positive 
            """
            #sample=random.sample(train_t[z],len(train_t[z])) #選全部的Positive
            """
            choose all YouTuber 
            """
            sample= all_auxilary #選全部的Positive
        
            #sample=random.sample(train_t[z]+train_f[z],len(train_t[z])+len(train_f[z]))
        
            #user degree of category favor 
            r_3=np.zeros(len(sample)) 
         
            for b in range(len(sample)):
                yes.append(all_3374[sample[b]])
                yesr.append(YouTuber_category[sample[b]]*user_category_norm[z])
        
            for b in range(len(yesr)):
                r_3[b]=max(yesr[b])
            #print('r_3:',r_3)
        
            yes=np.array(yes)
        
            #取positive 
            train_t_sample = random.sample(train_t[z],len(train_t[z]))
            #print('number of positive feedback', len(train_t_sample))
        
            #train_f_sample = random.sample(train_f[z],20)
            for ta in train_t_sample:
                pos = sample.index(ta)
                
                image_1=np.expand_dims(all_3374[ta],0) #(1,2048)
                train_f_sample = random.sample(train_f[z],10)
                
                for b in train_f_sample:
                    image_2=np.expand_dims(all_3374[b],0) #(1,2048)
                    _last_be_relu,_norm_par,_a_list,r3,_auc, _loss,_=sess.run([last_be_relu,norm_par,a_list_smooth,a_list_soft,auc,loss,train_op], feed_dict={user: [z],
                                        i: [ta], j: [b], xf: yes , l_id:sample, l_id_len:[len(sample)],positive_id:train_t[z],positive_len:[len(train_t[z])],r:r_3,
                                        image_i:image_1,image_j:image_2})
                    #print(_a_list)
                    #print(r3)
                    train_auc+=_auc
                    total_loss+=_loss
                    length += 1
        #print('a_list:',_a_list)
        #print('a_list_soft:',r3)
        print("total_loss:-----------------", total_loss/length)
        print("train_auc:-------------------", train_auc/length)
        loss_acc_list.append([total_loss/length,train_auc/length,time.time()-t0])
        print('time:',time.time()-t0,' sec')
    print('Total cost ',time.time()-t0,' sec')   
    U, Y, A, E, Au, Ay, Aa, Av,B =sess.run([user_latent, item_latent, aux_item, embedding, Wu, Wy, Wa, Wv,Beta])
    np.savez('./Result/Cold_user/'+save_name+'.npz', 
                        U=U, Y=Y, A=A, E=E, Wu=Au, Wy=Ay, Wa=Aa, Wv=Av,B=B)
    return U, Y, A, E, Au, Ay, Aa, Av,B

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]

no_feature = 'Cold_user_all_y57_'
#all_3374 = np.load('../Data/npy/mask_feature/'+no_feature+'.npy')
#l = 2939-2048
try_count = [str(try_i+1) for try_i in range(19)]

testcount = 0
finish_list = []
for try_i in try_count:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished count',finish_list)
                    finish_list.append(try_i)
                    print('Now :',try_i)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(no_feature+try_i+'_Edims200')
                    print('Finish dims:,',try_i)

Finished count []
Now : 1
Iteraction: 0


In [ ]:
# YouTuber All 
all_3374 = np.load('all_2939D.npy')
YouTuber_category = np.load('YouTuber_category_0.7.npy')
user_test_amount = int(user_following.shape[0]*0.2)
yt_test_amount = 10
yt_num = user_following.shape[1]

In [ ]:
train_t,train_f,test_t,test_f,user_category_norm = generate_train_test(user_following,all_3374,user_category,YouTuber_category,active_users,user_test_amount,yt_test_amount)

In [ ]:
#average num of following for training user
total_train = 0
for t in train_t:
    total_train += len(t)
avg = total_train/len(user_following)
print('training',avg)
#average num of following for testing user
total_test = 0
for t in test_t:
    total_test += len(t)
avg = total_test/user_test_amount
print('testing',avg)
print('比例:',avg/yt_test_amount)

In [ ]:
all_auxilary = [i for i in range(yt_num)]
n = len(user_following)
m = yt_num  
k = 64
l = all_3374.shape[1]
#embedding_dims = 150

In [ ]:
par_weights = [0.01]
beta_weights = [0.001]
Embedding_weights = [0.01]
Embedding_dims = [200]

no_feature = 'Cold_user_all_y88_'
#all_3374 = np.load('../Data/npy/mask_feature/'+no_feature+'.npy')
#l = 2939-2048
try_count = [str(try_i) for try_i in range(20)]

testcount = 0
finish_list = []
for try_i in try_count:
    for pary_weight in par_weights:
        for beta_weight in beta_weights:
            for Embedding_weight in Embedding_weights:
                for embedding_dims in Embedding_dims:
                    clear_output()
                    print('Finished count',finish_list)
                    finish_list.append(try_i)
                    print('Now :',try_i)
                    """
                    n: the number of users
                    m: the number of YouTubers
                    k: latent dims
                    l: feature dims
                    """
                    tf.reset_default_graph()

                    user = tf.placeholder(tf.int32,shape=(1,))
                    i = tf.placeholder(tf.int32, shape=(1,))
                    j = tf.placeholder(tf.int32, shape=(1,))

                    #多少個auxliary 
                    xf = tf.placeholder(tf.float32, shape=(None,l))
                    l_id = tf.placeholder(tf.int32, shape=(None,))
                    l_id_len = tf.placeholder(tf.int32,shape=(1,))
                    positive_id = tf.placeholder(tf.int32, shape=(None,))
                    positive_len = tf.placeholder(tf.int32,shape=(1,))
                    r = tf.placeholder(tf.float32,shape=(None,))


                    image_i = tf.placeholder(tf.float32, shape=(1,l))
                    image_j = tf.placeholder(tf.float32, shape=(1,l))

                    with tf.variable_scope("item_level"):
                        user_latent = tf.get_variable("user_latent", [n, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        item_latent = tf.get_variable("item_latent", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3)) 
                        aux_item = tf.get_variable("aux_item", [m, k],
                                                              initializer=tf.random_normal_initializer(0,0.1,seed=3))
                        Wu = tf.get_variable("Wu", [n,m,k],  
                                                              initializer=tf.contrib.layers.xavier_initializer())
                        Wy = tf.get_variable("Wy", [n,m,k],   
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wa = tf.get_variable("Wa", [n,m,k],  
                                                             initializer=tf.contrib.layers.xavier_initializer())
                        Wv = tf.get_variable("Wv", [n,m,embedding_dims],  
                                                            initializer=tf.contrib.layers.xavier_initializer())
                        #Wve =  tf.get_variable("Wve", [embedding_dims,l],  
                        #                                    initializer=tf.contrib.layers.xavier_initializer())

                        #每個user 對於每個YouTuber都有一個權重
                        #w1拿掉，wu
                        #hyper?

                        aux_new = tf.get_variable("aux_new", [1,k], initializer=tf.constant_initializer(0.0))
                        ########## Error part, how to get auxisize dynamically
                        ####aux_size= tf.get_variable(name='aux_size', initializer=l_id.get_shape().as_list()[-1])

                    with tf.variable_scope('feature_level'):
                        embedding = tf.get_variable("embedding", [embedding_dims,l],
                                    initializer=tf.contrib.layers.xavier_initializer())
                        Beta = tf.get_variable("beta", [n,embedding_dims],
                                    initializer=tf.random_normal_initializer(0.01,0.001,seed=10))

                    #lookup the latent factors by user and id
                    u = tf.nn.embedding_lookup(user_latent, user) #(1*k) 第幾個user latent factor
                    vi = tf.nn.embedding_lookup(item_latent, i) 
                    vj = tf.nn.embedding_lookup(item_latent, j)


                    wu = tf.squeeze(tf.nn.embedding_lookup(Wu, user)) #(m*k)
                    wy = tf.squeeze(tf.nn.embedding_lookup(Wy, user)) #(m*k)
                    wa = tf.squeeze(tf.nn.embedding_lookup(Wa, user)) #(m*k)
                    wv = tf.squeeze(tf.nn.embedding_lookup(Wv, user)) #(m,l)
                    beta = tf.nn.embedding_lookup(Beta, user) #user feature latent factor


                    a_list=tf.Variable([])
                    q = tf.constant(0)
                    def att_cond(q,a_list):
                        return tf.less(q,l_id_len[0])
                    def att_body(q,a_list):
                        xfi = tf.expand_dims(xf[q],0) #(1,l)
                        wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[q]),0) #取該YOUTUBER那欄(1,K)
                        a_list = tf.concat([a_list,[(tf.nn.relu( tf.matmul(wuui, u, transpose_b=True) +
                                tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[q]),0), transpose_b=True) +
                                tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0), transpose_b=True) +
                                tf.matmul(wvui,tf.matmul(embedding,xfi, transpose_b=True)))[0][0])*r[q]]],0)
                        q += 1
                        return q,  a_list

                    _, a_list = tf.while_loop(att_cond,att_body,[q,a_list],shape_invariants=[q.get_shape(),tf.TensorShape([None])])

                    # for while for smoothing
                    #a_list_soft=tf.nn.softmax(a_list)
                    a_list_smooth = tf.add(a_list,0.0000000001)
                    a_list_soft = tf.divide(a_list_smooth,tf.reduce_sum(a_list_smooth, 0)) #without softmax

                    norm_par = [wu,wy,wa,wv]

                    wuui = tf.expand_dims(tf.nn.embedding_lookup(wu,l_id[-1]),0)
                    wyui = tf.expand_dims(tf.nn.embedding_lookup(wy,l_id[-1]),0)
                    waui = tf.expand_dims(tf.nn.embedding_lookup(wa,l_id[-1]),0)
                    wvui = tf.expand_dims(tf.nn.embedding_lookup(wv,l_id[-1]),0)
                    wu_be_relu = tf.matmul(wuui, u, transpose_b=True)
                    wy_be_relu = tf.matmul(wyui, tf.expand_dims(tf.nn.embedding_lookup(item_latent,l_id[-1]),0), transpose_b=True)
                    wa_be_relu = tf.matmul(waui, tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[-1]),0), transpose_b=True)
                    wv_be_relu = tf.matmul(wvui, tf.matmul(embedding,tf.expand_dims(xf[-1],0), transpose_b=True))
                    last_be_relu = [wu_be_relu,wy_be_relu,wa_be_relu,wv_be_relu]

                    aux_np = tf.expand_dims(tf.zeros(k),0) #dimension (1,32)
                    q = tf.constant(0)
                    def sum_att_cond(q,aux_np):
                        return tf.less(q,l_id_len[0])

                    def sum_att_body(q,aux_np):
                        #aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                        aux_np = tf.math.add_n([aux_np,a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)]) 
                        q += 1
                        return q, aux_np

                    _,aux_np = tf.while_loop(sum_att_cond,sum_att_body,[q,aux_np])

                    """
                    for q in range(3): #取q個auxliary item
                        aux_np+=a_list_soft[q]*tf.expand_dims(tf.nn.embedding_lookup(aux_item, l_id[q]),0)
                    """

                    aux_part = tf.matmul(aux_np, vi, transpose_b=True)
                    #tf.print('aux attention:',aux_np)
                    aux_np+=u #user_latent factor + sum (alpha*auxilary)
                    aux_new=tf.assign(aux_new,aux_np) #把aux_new 的 值變成aux_np


                    latent_i_part = tf.matmul(aux_new, vi, transpose_b=True)
                    feature_i_part = tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    latent_j_part = tf.matmul(aux_new, vj, transpose_b=True)
                    feature_j_part = tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))
                    only_aux_i_part = tf.matmul(aux_np, vi, transpose_b=True)
                    only_aux_j_part = tf.matmul(aux_np, vj, transpose_b=True)

                    #矩陣中對應函數各自相乘
                    # ex: tf.matmul(thetav,(tf.matmul(embedding, image_i, transpose_b=True)))
                    xui = tf.matmul(aux_new, vi, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_i, transpose_b=True)))
                    xuj = tf.matmul(aux_new, vj, transpose_b=True)+ tf.matmul(beta,(tf.matmul(embedding,image_j, transpose_b=True)))

                    xuij = tf.subtract(xui,xuj)


                    l2_norm = tf.add_n([
                                0.0001 * tf.reduce_sum(tf.multiply(u, u)),
                                0.0001 * tf.reduce_sum(tf.multiply(vi, vi)),
                                0.0001 * tf.reduce_sum(tf.multiply(vj, vj)),


                                0.01 * tf.reduce_sum(tf.multiply(wu, wu)),
                                pary_weight * tf.reduce_sum(tf.multiply(wy, wy)),
                                pary_weight * tf.reduce_sum(tf.multiply(wa, wa)),
                                pary_weight * tf.reduce_sum(tf.multiply(wv,wv)),

                                beta_weight * tf.reduce_sum(tf.multiply(beta,beta)),
                                Embedding_weight * tf.reduce_sum(tf.multiply(embedding,embedding)),

                                ])

                    loss = l2_norm -tf.log(tf.sigmoid(xuij)) # objective funtion
                    train_op = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss) #parameter optimize 
                    auc = tf.reduce_mean(tf.to_float(xuij > 0))

                    Ur, Yr, Ar, Er, Aur, Ayr, Aar, Avr,Br = training(no_feature+try_i+'_Edims200')
                    print('Finish dims:,',try_i)

# Cold YouTuber